The code in this document was written as part of the final project for the Spring 2021 offering of ECON46.

We begin by importing all the requisite packages.

In [ ]:
from pandas_datareader._utils import RemoteDataError
import matplotlib.pyplot as plt
import pandas_datareader as dr
import pandas as pd
import networkx as nx
import seaborn as sns
import numpy as np
import datetime
import math

In [ ]:
globalList = []

The following function returns a list of ETF holdings given the name of a fund.

In [ ]:
def tickersOfETFHoldings(etf_ticker):
  url = 'http://research2.fidelity.com/fidelity/screeners/etf/public/etfholdings.asp?symbol={}&view=Region'.format(etf_ticker)
  hd = pd.read_html(url)[0]
  hd = hd.query('Weight >= 0.5 & Geography == "United States"')
  initial = hd["Symbol"].tolist()
  result = [i for i in initial if (isinstance(i, str)) and i.isalpha()]
  return result

The following function produces a Pandas DataFrame of ticker prices between specified `start` and `end` dates when provided with a list of stock tickers.



In [ ]:
def makeDFfromTickers(tickers, start, end):
  result = pd.DataFrame()
  for ticker in tickers:
    try:
      history = dr.DataReader(ticker, 'yahoo', start, end)
      result[ticker] = pd.Series(history["Adj Close"])
    except (RemoteDataError, TypeError, NameError, KeyError):
      pass
  return result

The following function returns the vertex with highest betweenness centrality when given a NetworkX graph.

In [ ]:
def getBetweenness(graph):
  return max(nx.betweenness_centrality(graph), key=nx.betweenness_centrality(graph).get)

The following function returns the difference in price of a stock between two dates `start` and `end`.

In [ ]:
def change(symbol, start, end):
  try:
    data = dr.DataReader(symbol, 'yahoo', start, end)
    change = ((float(data["Adj Close"][-1]) - float(data["Adj Close"][0]))/float(data["Adj Close"][0])) * 100
  except (RemoteDataError, TypeError, NameError, KeyError):
    change = "Couldn't get the difference"
  return change

The following function almost replicates the functionality of the `change()` function written above, but is used as a utility to print the difference directly to the cell output.

In [ ]:
def price_between_dates(symbol, start, end):
  answer = ""
  try:
    data = dr.DataReader(symbol, 'yahoo', start, end)
    change = ((float(data["Adj Close"][-1]) - float(data["Adj Close"][0]))/float(data["Adj Close"][0])) * 100
    answer = "Between {start} and {end}, the price of {symbol} changed by {change}%.".format(start=start, end=end, symbol=symbol, change=change)
  except (RemoteDataError, TypeError, NameError, KeyError):
    pass
  return answer

This is the main simulation function that produces a stock correlation matrix and finds the vertex with highest betweenness centrality.

In [ ]:
def doThingETF(etfSymbol, start, end, normalized=pd.DataFrame()):
  tickers = tickersOfETFHoldings(etfSymbol)
  prices = makeDFfromTickers(tickers, start, end)

  for column in prices.columns:
      normalized[column] = np.log(prices[column]).diff(-1)

  stock_correlation_matrix = normalized.corr()

  edges = stock_correlation_matrix.stack()
  edges = edges.reset_index()
  edges.columns = ["first", "second", "corr"]
  edges = edges.loc[edges["first"] != edges["second"]]
  edges = edges.copy()

  threshold, under_thresh = 0.5, []
  G = nx.from_pandas_edgelist(edges, "first", "second", edge_attr=["corr"])
  for edge in G.edges():
      first, second = edge
      correlation = G[first][second]["corr"]
      if abs(correlation) < threshold:
          under_thresh.append((first, second))
  G.remove_edges_from(under_thresh)

  central = getBetweenness(G)

  try:
    if change(central, start, end) >= change(etfSymbol, start, end):
      globalList.append((etfSymbol, central))
  except (RemoteDataError, TypeError, NameError, KeyError):
    pass
  
  print("We are operating on {}.".format(etfSymbol))
  print("The most central stock is {central}, and the most heavily weighted American stock is {big}.".format(central=central, big=tickers[0]))
  print(price_between_dates(central, start, end))
  print(price_between_dates(etfSymbol, start, end))
  print('\n')

The following cell contains the body of the simulation. The results are printed to the cell output.

In [ ]:
etfs = ['ICLN', 'NLR', 'XOP', 'XLF', 'VFH', 'KCE', 'VIS', 'IYJ', 'ITA', 'WBIF']
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 31)
for etf in etfs:
  doThingETF(etf, start, end)
print(globalList)

etf_symbols = [i[0] for i in globalList]
stock_symbols = [i[1] for i in globalList]
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2020, 12, 31)
total = 0
for i in range(len(etf_symbols)):
  if change(stock_symbols[i], start, end) >= change(etf_symbols[i], start, end):
    print("{} satisfies the hypothesis.".format(etf_symbols[i]))
    print(price_between_dates(stock_symbols[i], start, end))
    print(price_between_dates(etf_symbols[i], start, end))
    total += 1
  else:
    print("{} doesn't satisfy the hypothesis".format(etf_symbols[i]))
  print('\n')

success = total/len(etfs)

print("Of the {numETFs} ETFs that you tested, {lenGlobList} were good. \
Of those, {total} satisfied the hypothesis. This is a success rate of \
{success}.".format(numETFs=len(etfs), lenGlobList=len(globalList), total=total, success=success))

The following function visualizes a stock correlation network; much of the code is copied from `doThingETF()` above.

In [ ]:
def makeETFgraph(etfSymbol, start, end, normalized=pd.DataFrame()):
  tickers = tickersOfETFHoldings(etfSymbol)
  prices = makeDFfromTickers(tickers, start, end)

  for column in prices.columns:
      normalized[column] = np.log(prices[column]).diff(-1)
  stock_correlation_matrix = normalized.corr()

  edges = stock_correlation_matrix.stack()
  edges = edges.reset_index()
  edges.columns = ["first", "second", "corr"]
  edges = edges.loc[edges["first"] != edges["second"]]
  edges = edges.copy()

  threshold, under_thresh = 0.5, []
  G = nx.from_pandas_edgelist(edges, "first", "second", edge_attr=["corr"])
  for edge in G.edges():
      first, second = edge
      correlation = G[first][second]["corr"]
      if abs(correlation) < threshold:
          under_thresh.append((first, second))
  G.remove_edges_from(under_thresh)
  
  node_size = []
  
  for key, value in dict(G.degree).items():
      node_size.append((45 * value) + 1)

  sns.set(rc={"figure.figsize": (7, 7)})

  nx.draw(G, pos=nx.circular_layout(G), with_labels=True, node_size=node_size, node_color="#5cc8ff", edge_color="#9792e3")
  plt.show()

Below is the command to produce a network visualization of the ICLN ETF based on stock correlation data during 2019.

In [ ]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 31)
makeETFgraph('ICLN', start, end)

The following cell runs a command using an open-source package to download this Google Colab notebook as a PDF; this is not part of the final project.

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('ECON 46 - Final Project - Nikesh Mishra.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.





[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/ECON 46 - Final Project - Nikesh Mishra.ipynb to pdf
[NbConvertApp] Writing 45594 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 37040 bytes to /content/drive/My Drive/ECON 46 - Final Project - Nikesh Mishra.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'